# Wasserstein Generative Adversarial Networks (WGAN)

Neste notebook, vamos explorar a Wasserstein GAN (WGAN), uma evolução significativa das Redes Adversariais Generativas que visa resolver alguns dos problemas mais persistentes do treinamento de GANs tradicionais, como a instabilidade e o colapso de modo. A principal inovação da WGAN é a substituição da divergência Jensen-Shannon (JS), implícita na função de custo da GAN original, pela Distância de Wasserstein-1, também conhecida como "Earth Mover's Distance". Isso resulta em uma função de perda que se correlaciona melhor com a qualidade das imagens geradas e proporciona gradientes mais estáveis, tornando o treinamento mais robusto. Construiremos nossa WGAN sobre a arquitetura convolucional da DCGAN, modificando a função de perda, a arquitetura do "Crítico" e o algoritmo de treinamento.

### A Distância de Wasserstein

O treinamento de GANs tradicionais é notoriamente instável. Isso se deve em grande parte à função de perda baseada na divergência JS, que pode saturar facilmente, resultando em gradientes que desaparecem (*vanishing gradients*). Quando o Discriminador se torna muito bom, o gradiente para o Gerador vai a zero, e o aprendizado para.

A WGAN propõe o uso da Distância de Wasserstein-1 ($W(P_r, P_g)$), que mede a "distância" entre a distribuição de dados reais ($P_r$) e a distribuição de dados gerados ($P_g$). Intuitivamente, pode ser vista como o "custo" mínimo para transformar uma distribuição na outra, como o custo de mover uma pilha de terra para que ela assuma a forma de outra.

$$
W(P_r, P_g) = \inf_{\gamma \in \Pi(P_r, P_g)} \mathbb{E}_{(x, y) \sim \gamma} [\|x - y\|]
$$

Esta formulação é intratável. No entanto, a dualidade de Kantorovich-Rubinstein nos oferece uma forma alternativa e mais prática:

$$
W(P_r, P_g) = \sup_{\|f\|_L \le 1} \mathbb{E}_{x \sim P_r}[f(x)] - \mathbb{E}_{x \sim P_g}[f(x)]
$$

Aqui, o supremo é obtido sobre todas as funções 1-Lipschitz $f$. Uma função é K-Lipschitz se $|f(x_1) - f(x_2)| \le K|x_1 - x_2|$. Na prática, a WGAN parametriza a função $f$ com uma rede neural, que chamamos de **Crítico** (em vez de Discriminador). O trabalho do Crítico é encontrar uma função $f$ que maximize a diferença acima. O trabalho do Gerador é produzir amostras que minimizem essa mesma diferença.

Para forçar a restrição de Lipschitz, o artigo original da WGAN propõe uma solução simples: o **weight clipping**. Após cada atualização de gradiente, os pesos do Crítico são "clipados" para um pequeno intervalo, como $[-0.01, 0.01]$.

In [ ]:
import torch
from torch import nn
from torch.optim import Adam, RMSprop
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Preparação dos Dados (MNIST)

Para treinar nossa DCGAN, utilizaremos o dataset MNIST. As imagens serão redimensionadas para 32x32 para facilitar uma arquitetura com múltiplas camadas de convolução transposta que dobram a dimensão espacial. Além disso, normalizaremos os pixels das imagens para o intervalo `[-1, 1]`, que corresponde à faixa da função de ativação `Tanh` na camada de saída do nosso Gerador.

In [ ]:
batch_size = 128
image_size = 32

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # Normaliza para [-1, 1]
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Número de batches de treino: {len(train_dataloader)}")
print(f"Número de batches de teste: {len(test_dataloader)}")

### O Gerador

A arquitetura do Gerador pode ser mantida exatamente a mesma da DCGAN. Sua função ainda é mapear um vetor do espaço latente para o espaço de imagens. A inovação da WGAN não está na arquitetura dos modelos, mas sim na forma como eles são treinados.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, features_g):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, features_g * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, img_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.net(input)

### O Crítico (Critic)

A arquitetura do Discriminador da DCGAN é reaproveitada, mas com uma mudança: a camada final de ativação `Sigmoid` é **removida**. O modelo, que agora chamamos de Crítico, não deve produzir uma probabilidade, mas sim um score (um número real) para cada imagem. Esse score é usado para aproximar a Distância de Wasserstein.

In [ ]:
class Critic(nn.Module):
    def __init__(self, img_channels, features_d):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, features_d, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # A camada final não tem Sigmoid, produz um score
            nn.Conv2d(features_d * 4, 1, kernel_size=4, stride=1, padding=0, bias=False),
        )

    def forward(self, input):
        return self.net(input)

### Inicialização e Instanciação dos Modelos

Instanciamos os modelos e aplicamos a mesma inicialização de pesos da DCGAN. A mudança principal aqui é o otimizador: o artigo da WGAN recomenda o uso do `RMSprop` em vez de Adam com momento, pois observaram maior estabilidade. Não precisamos mais de uma função de perda como `BCELoss`, pois o custo será calculado diretamente a partir dos scores do Crítico.

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

features_g = 16
latent_dim = 100
img_channels = 1

# Instanciação dos modelos
generator = Generator(latent_dim, img_channels, features_g=64).to(device)
generator.apply(weights_init)

critic = Critic(img_channels, features_d=64).to(device)
critic.apply(weights_init)

print("Generator Architecture:\n", generator)
print("\nCritic Architecture:\n", critic)

In [ ]:
lr = 0.00005

# Otimizadores (RMSprop é recomendado para WGAN)
optimizer_g = RMSprop(generator.parameters(), lr=lr)
optimizer_c = RMSprop(critic.parameters(), lr=lr)

### Loop de Treinamento

O loop de treinamento da WGAN é diferente do de uma GAN padrão. Para cada atualização do Gerador, o Crítico é atualizado múltiplas vezes (`n_critic`). Isso é feito para garantir que o Crítico se aproxime bem da função 1-Lipschitz ótima.

1.  **Treinamento do Crítico**:
    * A perda do Crítico visa maximizar $f(x) - f(G(z))$. Como otimizadores fazem minimização, minimizamos o negativo: $-(f(x) - f(G(z)))$, que é $f(G(z)) - f(x)$.
    * Após a atualização dos pesos, aplicamos o **weight clipping** para forçar a restrição de Lipschitz.

2.  **Treinamento do Gerador**:
    * A perda do Gerador visa minimizar a Distância de Wasserstein, o que corresponde a maximizar o score que o Crítico dá para suas imagens falsas, $f(G(z))$. Portanto, minimizamos $-f(G(z))$.

In [ ]:
num_epochs = 10
n_critic = 5 # Número de vezes que o crítico é treinado por iteração do gerador
clip_value = 0.01 # Valor para o weight clipping
plot_interval = 1

g_losses = []
c_losses = []

for epoch in range(num_epochs):
    generator.train()
    critic.train()
    
    g_running_loss = 0.0
    c_running_loss = 0.0

    for i, (real_imgs, _) in enumerate(train_dataloader):
        real_imgs = real_imgs.to(device)
        b_size = real_imgs.size(0)

        # --- Treinamento do Crítico (n_critic iterações) ---
        for _ in range(n_critic):
            optimizer_c.zero_grad()
            
            noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
            fake_imgs = generator(noise).detach()
            
            critic_real = critic(real_imgs).view(-1).mean()
            critic_fake = critic(fake_imgs).view(-1).mean()
            
            # A perda do crítico é -(score_real - score_fake)
            loss_c = -(critic_real - critic_fake)
            loss_c.backward()
            optimizer_c.step()

            # Clip weights of critic
            for p in critic.parameters():
                p.data.clamp_(-clip_value, clip_value)
        
        c_running_loss += loss_c.item()
        
        # --- Treinamento do Gerador ---
        optimizer_g.zero_grad()
        
        noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
        gen_imgs = generator(noise)
        output = critic(gen_imgs).view(-1).mean()
        
        # A perda do gerador é -score_fake
        loss_g = -output
        loss_g.backward()
        optimizer_g.step()

        g_running_loss += loss_g.item()

    g_epoch_loss = g_running_loss / len(train_dataloader)
    c_epoch_loss = c_running_loss / len(train_dataloader)
    
    g_losses.append(g_epoch_loss)
    c_losses.append(c_epoch_loss)

    if epoch % plot_interval == 0 or epoch == num_epochs - 1:
        print(f"[{epoch}/{num_epochs-1}] Loss C: {c_epoch_loss:.4f} | Loss G: {g_epoch_loss:.4f}")
        
        generator.eval()
        with torch.no_grad():
            n_images = 8
            sample_noise = torch.randn(n_images, latent_dim, 1, 1, device=device)
            generated_imgs = generator(sample_noise).detach().cpu()

            fig, axs = plt.subplots(2, 4, figsize=(8, 4))
            fig.suptitle(f'Imagens Geradas na Época {epoch}', fontsize=16)
            for i in range(n_images):
                row = i // 4
                col = i % 4
                axs[row, col].imshow(generated_imgs[i].squeeze(), cmap="gray", vmin=-1, vmax=1)
                axs[row, col].axis("off")
            plt.show()

### Análise dos Resultados

Uma das vantagens da WGAN é que a perda do Crítico ($f(G(z)) - f(x)$) se aproxima da Distância de Wasserstein, que tende a se correlacionar com a qualidade das imagens geradas. Ao contrário da perda de uma GAN tradicional, aqui, uma perda do Crítico menor (mais negativa) indica que a distância entre as distribuições real e gerada é maior.

In [ ]:
# Plotar o gráfico de perdas
plt.figure(figsize=(10, 5))
plt.title("Critic and Generator Loss During Training")
plt.plot(c_losses, label="Critic Loss")
plt.plot(g_losses, label="Generator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()